# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2016-09-01	Evaluating Different Strategies for Reduction of Mutation Testing Costs	1st Brazilian Symposium on Systematic and Automated Software Testing (SAST)	Mutation testing presents high efficacy in terms of revealed faults, but with high computational costs, because test cases must be executed against a great number of mutants. To reduce such costs several strategies exist. In general, they select a reduced number of mutants that maintain a great overall mutation score. Recently, Higher-Order Mutation Testing (HOM) was proposed. HOM introduces more than one fault at a time in the mutants such that they are harder to kill. This kind of testing can also be used as a cost reduction strategy. However, few works in literature evaluate the cost reduction potential of HOM or compare it to other existing strategies. In this work, we evaluate HOM as a mutation testing cost reduction strategy. We conducted an experiment to compare four

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [37]:
publications = pd.read_csv("publications.csv", sep=";", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,doi,paper_url_rg
0,2016-09-01,Evaluating Different Strategies for Reduction ...,Proceedings of the 1st Brazilian Symposium on ...,Mutation testing presents high efficacy in ter...,"J. A. Prado Lima, G. Guizzo, S. R. Vergilio, A...",2016-09-SAST,https://dl.acm.org/citation.cfm?id=2993292,10.1145/2993288.2993292,https://www.researchgate.net/publication/30805...
1,2017-09-20,Automatic Generation of Search-Based Algorithm...,Proceedings of the 31st Brazilian Symposium on...,The selection of products for the variability ...,"Helson L. Jakubovski Filho, Jackson A. Prado L...",2017-09-SBES,https://dl.acm.org/citation.cfm?doid=3131151.3...,10.1145/3131151.3131152,https://www.researchgate.net/publication/31989...
2,2017-09-19,A Multi-objective optimization approach for se...,Proceedings of the 2nd Brazilian Symposium on ...,The use of Higher-Order Mutants (HOMs) present...,"J. A. Prado Lima, S. R. Vergilio, A Multi-obje...",2017-09-SAST,https://dl.acm.org/citation.cfm?doid=3128473.3...,10.1145/3128473.3128479,https://www.researchgate.net/publication/31943...
3,2016-08-01,Deriving Products for Variability Test of Feat...,Applied Soft Computing,Deriving products from a Feature Model (FM) fo...,"A. Strickler, J. A. Prado Lima, S. R. Vergilio...",2016-05-ASOC,https://www.sciencedirect.com/science/article/...,10.1016/j.asoc.2016.07.059,https://www.researchgate.net/publication/30608...
4,2017-05-01,Hyper-Heuristic Based Product Selection for So...,IEEE Computational Intelligence Magazine,A Software Product Line (SPL) is defined as a ...,"T. N. Ferreira, J. A. Prado Lima, A. Strickler...",2017-05-IEEECIM,https://ieeexplore.ieee.org/document/7895294/,10.1109/MCI.2017.2670461,https://www.researchgate.net/publication/31646...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [38]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [39]:
import os
for row, item in publications.iterrows():
    
    md_filename = item.url_slug + ".md"
    html_filename = item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
    md += "\nDOI: " + str(item.doi) + "\n"
    
    if len(str(item.excerpt)) > 5:
        md += "\nAbstract\n==="
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url_rg)) > 5:
        md += "\nSee this paper on [Research Gate <i class=\"ai ai-researchgate\"></i>](" + item.paper_url_rg + ")\n" 
        
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [40]:
!ls ../_publications/

2016-05-ASOC.md  2017-05-IEEECIM.md  2017-09-SBES.md
2016-09-SAST.md  2017-09-SAST.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).